# Quick Start V1

From: [Quick Start V1](https://python.langchain.com/v0.1/docs/get_started/quickstart/)

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

## LLM Chain

In [9]:
llm = ChatOpenAI()

In [6]:
aimsg1 = llm.invoke("hello")

In [12]:
aimsg1.content

'Hello! How can I assist you today?'

### Simple Prompting

In [13]:
# Not good response
aimsg2 = llm.invoke("how can langsmith help with testing?")

'Langsmith can help with testing in a few ways:\n\n1. Automated testing: Langsmith can create automated tests for your code, helping to ensure that new changes do not introduce bugs or regressions.\n\n2. Code review: Langsmith can review code changes and provide feedback on potential issues or areas for improvement.\n\n3. Performance testing: Langsmith can help to identify bottlenecks and inefficiencies in your code, allowing you to optimize performance.\n\n4. Security testing: Langsmith can help to identify potential security vulnerabilities in your code, helping to ensure that your application is secure.\n\nOverall, Langsmith can help to improve the quality and reliability of your code through testing and feedback.'

In [14]:
print(aimsg2.content)

Langsmith can help with testing in a few ways:

1. Automated testing: Langsmith can create automated tests for your code, helping to ensure that new changes do not introduce bugs or regressions.

2. Code review: Langsmith can review code changes and provide feedback on potential issues or areas for improvement.

3. Performance testing: Langsmith can help to identify bottlenecks and inefficiencies in your code, allowing you to optimize performance.

4. Security testing: Langsmith can help to identify potential security vulnerabilities in your code, helping to ensure that your application is secure.

Overall, Langsmith can help to improve the quality and reliability of your code through testing and feedback.


### Define Prompt Template


guide its response with a prompt template. Prompt templates convert raw user input to better input to the LLM.

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [27]:
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a world class technical documentation writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

### Create LLM Chain

In [28]:
chain = prompt | llm 
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a world class technical documentation writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x110e72a70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x110ea2530>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [29]:
aimsg3 = chain.invoke({"input": "how can langsmith help with testing?"})

In [2]:
print(aimsg3.content)

NameError: name 'aimsg3' is not defined

Convert to string

In [33]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [37]:
chain = prompt | llm | output_parser

In [38]:
aimsg4 = chain.invoke({"input": "how can langsmith help with testing?"})

In [46]:
print(aimsg4)

Langsmith can help with testing in several ways:

1. Test Automation: Langsmith can be used to automate testing processes by writing scripts to simulate user interactions with the software. This can help in running tests quickly and efficiently, saving time and effort.

2. Data Generation: Langsmith can be used to generate test data for various scenarios, helping to ensure comprehensive test coverage and uncover potential issues in the software.

3. Performance Testing: Langsmith can be used to simulate heavy loads on the software and measure its performance under stress. This can help in identifying performance bottlenecks and optimizing the software for better efficiency.

4. Regression Testing: Langsmith can be used to automate regression testing, ensuring that new updates or changes to the software do not introduce any unexpected bugs or issues.

5. Integration Testing: Langsmith can be used to automate integration testing by simulating interactions between different components of 

### Retrival chain

In [3]:
from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

#### Embedding Model

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x12654a170>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x1266392d0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

build our index:

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [7]:
documents

[Document(page_content='LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.U

#### Create Retrival Chain

**A retrieval chain:** this chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='Answer the following question based only on the provided context:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x13ff3d9f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x13ff3f7c0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

Run this ourselves by passing in documents directly:

In [56]:
from langchain_core.documents import Document

aimsg5 = document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

In [57]:
aimsg5

'Langsmith can help with testing by allowing you to visualize test results.'

From Full Document

In [60]:
aimsg6 = document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": documents
})

In [62]:
print(aimsg6)

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications using these datasets, upload test cases in bulk or create them on the fly, export test cases from application traces, and run custom evaluations to score test results. Additionally, LangSmith provides a comparison view for tracking and diagnosing regressions in test scores across multiple revisions of an application. The platform also offers a playground environment for rapid iteration and experimentation, logging every playground run for creating test cases or comparing with other runs. Finally, LangSmith supports beta testing to collect more data on how LLM applications perform in real-world scenarios, capture feedback from users, and annotate traces for evaluation criteria.


However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [13]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

**Invoke this chain**
- This returns a dictionary
- the response from the LLM is in the `answer` key

In [14]:
aimsg7 = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(aimsg7["answer"])


LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications using these datasets, and easily evaluate the results. Developers can upload test cases in bulk, create them on the fly, or export them from application traces. LangSmith also supports custom evaluations (LLM and heuristic based) to score test results. Additionally, LangSmith provides a comparison view for tracking and diagnosing regressions in test scores across multiple revisions of an application. The platform also offers a playground environment for rapid iteration and experimentation, allowing users to test out different prompts and models and log each run for further analysis.
